In [39]:
import requests
from pyvi import ViTokenizer
text = "Chính phủ đang trên đà phát triển#$Bóng đá"
num = "1#$2"
re = requests.post("http://localhost:8000/TextClassification", data={"text":text, 'id':num})
re.json()['size']

4

In [13]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.Builder() \
     .appName("articles") \
     .getOrCreate()
static = spark.read.parquet("hdfs://10.3.71.86:8020/Data/Logging/kinghub/app/2020-07-01")
dataSchema = static.schema

In [14]:
static.printSchema()

root
 |-- serverTime: long (nullable = true)
 |-- clientTime: long (nullable = true)
 |-- dt: string (nullable = true)
 |-- eventId: integer (nullable = true)
 |-- isVirtual: integer (nullable = true)
 |-- postId: string (nullable = true)
 |-- itemId: string (nullable = true)
 |-- cardType: integer (nullable = true)
 |-- contentType: integer (nullable = true)
 |-- extra: string (nullable = true)
 |-- osType: integer (nullable = true)
 |-- osVersion: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- deviceModel: string (nullable = true)
 |-- deviceBuildNumber: string (nullable = true)
 |-- deviceId: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- appId: string (nullable = true)
 |-- appVersion: string (nullable = true)
 |-- language: string (nullable = true)
 |-- networkStatus: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- pageId: long (nullable = true)
 |-- boxId: string (nullable = true)
 |-- tagId: string (nullable = true

In [15]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 100)\
.parquet("hdfs://10.3.71.86:8020/Data/Logging/kinghub/app/2020-07-01")

In [16]:
activityCounts = streaming.groupBy("itemId").count()
# activityCounts = streaming

In [17]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [19]:
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
.format("memory").outputMode("complete")\
.start()
# activityQuery.show()

In [ ]:
activityQuery.awaitTermination(timeout=10)

In [ ]:
list(spark.streams.active)

In [7]:
# activityQuery.awaitTermination()
from time import sleep
for x in range(5):
    
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

AnalysisException: Table or view not found: activity_counts; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [activity_counts]
